In [8]:
from pathlib import Path
import sys

proj_path = Path().resolve()

    
from aib9_lib import aib9_tools as aib9
import numpy as np
import torch
import os
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

Using device: mps


In [9]:
coords = np.load('molecule_coords.npy')

In [10]:
coords=coords[0]
view=aib9.py3dplot(coords, style = 'ball_stick')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
TOPO_FILE = (
   "/Users/chriszhang/Documents/aib9_vanillavae/visnet_aib9/aib9_lib/aib9_atom_info.npy"
) 
topo = np.load(TOPO_FILE)

In [16]:
topo

array([['CH3', '1'],
       ['C', '1'],
       ['O', '1'],
       ['N', '2'],
       ['CA', '2'],
       ['CB1', '2'],
       ['CB2', '2'],
       ['C', '2'],
       ['O', '2'],
       ['N', '3'],
       ['CA', '3'],
       ['CB1', '3'],
       ['CB2', '3'],
       ['C', '3'],
       ['O', '3'],
       ['N', '4'],
       ['CA', '4'],
       ['CB1', '4'],
       ['CB2', '4'],
       ['C', '4'],
       ['O', '4'],
       ['N', '5'],
       ['CA', '5'],
       ['CB1', '5'],
       ['CB2', '5'],
       ['C', '5'],
       ['O', '5'],
       ['N', '6'],
       ['CA', '6'],
       ['CB1', '6'],
       ['CB2', '6'],
       ['C', '6'],
       ['O', '6'],
       ['N', '7'],
       ['CA', '7'],
       ['CB1', '7'],
       ['CB2', '7'],
       ['C', '7'],
       ['O', '7'],
       ['N', '8'],
       ['CA', '8'],
       ['CB1', '8'],
       ['CB2', '8'],
       ['C', '8'],
       ['O', '8'],
       ['N', '9'],
       ['CA', '9'],
       ['CB1', '9'],
       ['CB2', '9'],
       ['C', '9'],
       [